In [1]:
#Cración de la sesión de Spark

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [44]:
#CARGA DE LOS ARCHIVOS

df = spark.read.options(inferSchema=True, multiLine=True, delimiter=",", header=True).csv("/Users/christian/Desktop/Trabajo Big Data/Copia de Pares de Palabras.csv")

In [76]:
#DESCRIPCIÓN BD

import pandas as pd
pddf = pd.read_csv("/Users/christian/Desktop/Trabajo Big Data/Copia de Pares de Palabras.csv")
pddf.describe()

,FrEstC,IDP,IDI,Ed,Esc,Gra,ApEs,FrEs,StEs,LEs,ApGen,FrGen,StGen,LGen,Face,FrFace,SF,LF,SFBool
count,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000
mean,7.137255,9.000000,251.150327,13.947712,6.666667,2.666667,0.183007,0.405229,0.002109,2479.137255,0.490196,5.862745,0.032448,1970.307190,0.509804,6.267974,0.023626,2407.666667,1.023626
std,3.967066,4.915068,187.038899,0.705158,1.891811,0.472953,0.387942,1.096873,0.116890,9253.799822,0.501546,14.906754,0.119126,8873.017305,0.501546,15.423413,0.103745,8803.154742,0.103745
min,2.000000,1.000000,3.000000,13.000000,4.000000,2.000000,0.000000,0.000000,-0.800000,0.000000,0.000000,0.000000,-0.116667,0.000000,0.000000,0.000000,-0.400000,0.000000,0.600000
25%,4.000000,5.000000,71.000000,13.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,6.000000,9.000000,242.000000,14.000000,8.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,8.000000,1.000000
75%,8.000000,13.000000,388.000000,14.000000,8.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,0.001429,226.000000,1.000000,4.000000,0.005471,486.000000,1.005471
max,22.000000,17.000000,605.000000,15.000000,8.000000,3.000000,1.000000,5.000000,0.800000,78757.000000,1.000000,81.000000,0.700000,78757.000000,1.000000,81.000000,0.500000,78757.000000,1.500000


In [4]:
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
'Modelo de Clasificación según Sentimientos en base a la Edad, por Palabra'

In [45]:
ratings = (df
    .select(
        'Ed',
        'IDP',
        'SFBool',
    )
).cache()

(training, test) = ratings.randomSplit([0.8, 0.2])

In [46]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=50, maxIter=2, regParam=0.01, 
          userCol="Ed", itemCol="IDP", ratingCol="SFBool",
          coldStartStrategy="drop",
          implicitPrefs=False)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="SFBool",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.18567336919062224


In [47]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.count()
# Generate top 10 user recommendations for each movie
palRecs = model.recommendForAllItems(10)
palRecs.count()

17

In [48]:
userRecs.orderBy('Ed').show(truncate = False)
palRecs.orderBy('IDP').show(truncate = False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Ed |recommendations                                                                                                                                                      |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|13 |[[3, 1.2174941], [13, 1.0329115], [8, 1.0271486], [12, 1.0158], [14, 1.0154381], [10, 1.0094692], [9, 1.0085348], [7, 1.0004789], [17, 0.9995919], [2, 0.9875559]]   |
|14 |[[3, 1.2101064], [2, 1.1073079], [5, 1.0955043], [8, 1.0680277], [11, 1.0518379], [13, 1.0432236], [14, 1.0290325], [9, 1.016835], [10, 1.0098805], [7, 1.0050422]]  |
|15 |[[3, 1.1790522], [2, 1.1694136], [14, 1.1562577], [13, 1.063871], [8, 1.0471141], [9, 1.0341772], [5, 1.0224502], [12, 0.9813835], [11,

In [ ]:
'Modelo de Clasificación según sentimientos por Escuela, en base a palabras'

In [49]:
ratings = (df
    .select(
        'Esc',
        'IDP',
        'SFBool',
    )
).cache()

(training, test) = ratings.randomSplit([0.8, 0.2])

In [55]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=10, maxIter=2, regParam=0.01, 
          userCol="Esc", itemCol="IDP", ratingCol="SFBool",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="SFBool",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.44929300514533516


In [56]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.count()
# Generate top 10 user recommendations for each movie
palRecs = model.recommendForAllItems(10)
palRecs.count()

17

In [57]:
userRecs.orderBy('Esc').show(truncate = False)
palRecs.orderBy('IDP').show(truncate = False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Esc|recommendations                                                                                                                                                      |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|4  |[[12, 1.4931872], [15, 1.4914114], [9, 1.4881017], [10, 1.487039], [6, 1.487039], [14, 1.4865375], [11, 1.469443], [17, 1.3382381], [1, 1.3346843], [5, 1.3308353]]  |
|8  |[[4, 1.7350956], [9, 1.7090315], [17, 1.7089168], [1, 1.7031779], [15, 1.6899009], [14, 1.6887505], [10, 1.6682867], [6, 1.6682867], [12, 1.6640316], [5, 1.6554633]]|
+---+---------------------------------------------------------------------------------------------------------------------------------------

In [58]:
ratings = (df
    .select(
        'Esc',
        'IDP',
        'SFBool',
    )
).filter('Gra = 3').cache()

(training, test) = ratings.randomSplit([0.8, 0.2])

In [64]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=10, maxIter=2, regParam=0.01, 
          userCol="Esc", itemCol="IDP", ratingCol="SFBool",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="SFBool",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.2744906229473031


In [65]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.count()
# Generate top 10 user recommendations for each movie
palRecs = model.recommendForAllItems(10)
palRecs.count()

17

In [66]:
userRecs.orderBy('Esc').show(truncate = False)
palRecs.orderBy('IDP').show(truncate = False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Esc|recommendations                                                                                                                                                      |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|4  |[[4, 1.5390342], [14, 1.4925904], [16, 1.488037], [5, 1.488037], [13, 1.484934], [17, 1.4781239], [15, 1.3369218], [10, 1.336067], [7, 1.336067], [1, 1.3329141]]    |
|8  |[[15, 1.5064856], [10, 1.4916141], [7, 1.4916141], [13, 1.4885006], [1, 1.4857652], [6, 1.4779576], [11, 1.473465], [14, 1.4680126], [4, 1.3422024], [16, 1.3392671]]|
+---+---------------------------------------------------------------------------------------------------------------------------------------

In [67]:
ratings = (df
    .select(
        'Esc',
        'IDP',
        'SFBool',
    )
).filter('Esc = 4 OR (Esc = 8 AND Gra = 2)').cache()

(training, test) = ratings.randomSplit([0.8, 0.2])

In [73]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=10, maxIter=2, regParam=0.01, 
          userCol="Esc", itemCol="IDP", ratingCol="SFBool",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="SFBool",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.429921465258756


In [74]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.count()
# Generate top 10 user recommendations for each movie
palRecs = model.recommendForAllItems(10)
palRecs.count()

17

In [75]:
userRecs.orderBy('Esc').show(truncate = False)
palRecs.orderBy('IDP').show(truncate = False)

+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Esc|recommendations                                                                                                                                                       |
+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|4  |[[15, 1.4960339], [16, 1.4924563], [1, 1.4892597], [17, 1.4890047], [7, 1.4854625], [11, 1.4772636], [4, 1.3845066], [12, 1.3435644], [13, 1.3321434], [5, 1.3315073]]|
|8  |[[4, 1.5465734], [15, 1.5106524], [16, 1.4986949], [1, 1.4985219], [2, 1.4904319], [12, 1.489423], [10, 1.4798886], [11, 1.479352], [13, 1.4786276], [5, 1.432838]]   |
+---+----------------------------------------------------------------------------------------------------------------------------------

In [ ]:
'Modelo de Clasificación según sentimientos por Grado, en base a palabras'

In [88]:
ratings = (df
    .select(
        'Gra',
        'IDP',
        'SFBool',
    )
).filter('Esc = 8').cache()

(training, test) = ratings.randomSplit([0.8, 0.2])

In [89]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=10, maxIter=2, regParam=0.01, 
          userCol="Gra", itemCol="IDP", ratingCol="SFBool",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="SFBool",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.2487545339395933


In [90]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.count()
# Generate top 10 user recommendations for each movie
palRecs = model.recommendForAllItems(10)
palRecs.count()

17

In [91]:
userRecs.orderBy('Gra').show(truncate = False)
palRecs.orderBy('IDP').show(truncate = False)

+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Gra|recommendations                                                                                                                                                    |
+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2  |[[4, 1.546942], [15, 1.5123147], [6, 1.5000292], [1, 1.4994777], [12, 1.4964045], [8, 1.494771], [2, 1.4879663], [13, 1.4863079], [11, 1.4710009], [17, 1.4583882]]|
|3  |[[15, 1.51473], [4, 1.5046874], [6, 1.4999218], [12, 1.498362], [1, 1.4946713], [8, 1.4946603], [13, 1.4904047], [10, 1.4891322], [7, 1.4891322], [16, 1.4629638]] |
+---+-------------------------------------------------------------------------------------------------------------------------------------------------

In [98]:
ratings = (df
    .select(
        'IDI',
        'IDP',
        'SFBool',
    )
).filter('Esc = 8').cache()

(training, test) = ratings.randomSplit([0.8, 0.2])

In [99]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=500, maxIter=2, regParam=0.01, 
          userCol="IDI", itemCol="IDP", ratingCol="SFBool",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="SFBool",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8165498445072347


In [100]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.count()
# Generate top 10 user recommendations for each movie
palRecs = model.recommendForAllItems(10)
palRecs.count()

17

In [101]:
userRecs.orderBy('IDI').show()
palRecs.orderBy('IDP').show(truncate = False)

+---+--------------------+
|IDI|     recommendations|
+---+--------------------+
|  3|[[1, 0.99863064],...|
| 10|[[16, 0.998956], ...|
| 18|[[5, 1.3316139], ...|
| 21|[[5, 0.9992255], ...|
| 24|[[2, 0.9988625], ...|
| 26|[[7, 1.3312584], ...|
| 27|[[4, 0.99906373],...|
| 32|[[10, 0.9989354],...|
| 33|[[5, 0.9990322], ...|
| 40|[[17, 1.3313593],...|
| 43|[[12, 0.99832755]...|
| 44|[[10, 0.9989354],...|
| 51|[[12, 0.99851894]...|
| 60|[[15, 0.99888945]...|
| 69|[[10, 0.9989354],...|
| 71|[[1, 0.99806035],...|
| 78|[[6, 0.9988905], ...|
| 81|[[10, 0.9989354],...|
| 85|[[2, 0.9988183], ...|
|105|[[12, 0.9990515],...|
+---+--------------------+
only showing top 20 rows

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|IDP|recommendations                                                                                                  